<a href="https://colab.research.google.com/github/Kawarjeet/Advanced-Python-for-Data-Science/blob/main/Assignment1/nbody_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
"""
    N-body simulation.
"""

def compute_deltas(x1, x2, y1, y2, z1, z2):
    return (x1-x2, y1-y2, z1-z2)
    
def compute_b(m, dt, dx, dy, dz):
    mag = compute_mag(dt, dx, dy, dz)
    return m * mag

def compute_mag(dt, dx, dy, dz):
    return dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

def update_vs(v1, v2, dt, dx, dy, dz, m1, m2):
    v1[0] -= dx * compute_b(m2, dt, dx, dy, dz)
    v1[1] -= dy * compute_b(m2, dt, dx, dy, dz)
    v1[2] -= dz * compute_b(m2, dt, dx, dy, dz)
    v2[0] += dx * compute_b(m1, dt, dx, dy, dz)
    v2[1] += dy * compute_b(m1, dt, dx, dy, dz)
    v2[2] += dz * compute_b(m1, dt, dx, dy, dz)

def update_rs(r, dt, vx, vy, vz):
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz

def advance(BODIES,dt):
    '''
        advance the system one timestep
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ([x1, y1, z1], v1, m1) = BODIES[body1]
                ([x2, y2, z2], v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                update_vs(v1, v2, dt, dx, dy, dz, m1, m2)
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        update_rs(r, dt, vx, vy, vz)

def compute_energy(m1, m2, dx, dy, dz):
    return (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
    
def report_energy(BODIES,e=0.0):
    '''
        compute the energy and return it so that it can be printed
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ((x1, y1, z1), v1, m1) = BODIES[body1]
                ((x2, y2, z2), v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                e -= compute_energy(m1, m2, dx, dy, dz)
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(BODIES,ref, px=0.0, py=0.0, pz=0.0):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up local state
    BODIES = {
        'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], 4 * 3.14159265358979323 * 3.14159265358979323),

        'jupiter': ([4.84143144246472090e+00,
                     -1.16032004402742839e+00,
                     -1.03622044471123109e-01],
                    [1.66007664274403694e-03 * 365.24,
                     7.69901118419740425e-03 * 365.24,
                     -6.90460016972063023e-05 * 365.24],
                    9.54791938424326609e-04 * 4 * 3.14159265358979323 * 3.14159265358979323),

        'saturn': ([8.34336671824457987e+00,
                    4.12479856412430479e+00,
                    -4.03523417114321381e-01],
                   [-2.76742510726862411e-03 * 365.24,
                    4.99852801234917238e-03 * 365.24,
                    2.30417297573763929e-05 * 365.24],
                   2.85885980666130812e-04 * 4 * 3.14159265358979323 * 3.14159265358979323),

        'uranus': ([1.28943695621391310e+01,
                    -1.51111514016986312e+01,
                    -2.23307578892655734e-01],
                   [2.96460137564761618e-03 * 365.24,
                    2.37847173959480950e-03 * 365.24,
                    -2.96589568540237556e-05 * 365.24],
                   4.36624404335156298e-05 * 4 * 3.14159265358979323 * 3.14159265358979323),

        'neptune': ([1.53796971148509165e+01,
                     -2.59193146099879641e+01,
                     1.79258772950371181e-01],
                    [2.68067772490389322e-03 * 365.24,
                     1.62824170038242295e-03 * 365.24,
                     -9.51592254519715870e-05 * 365.24],
                    5.15138902046611451e-05 * 4 * 3.14159265358979323 * 3.14159265358979323)}
    offset_momentum(BODIES,BODIES[reference])

    for _ in range(loops):
        e = report_energy(BODIES)
        for _ in range(iterations):
            advance(BODIES,0.01)
        print(e)

if __name__ == '__main__':
    nbody(100, 'sun', 20000)

-0.1690751638285245
-0.16908926275527172
-0.1690524049239141
-0.16901474404530112
-0.16902307738080638
-0.1690798593916632
-0.16908378513402958
-0.16904612370294547
-0.16901416322875817
-0.1690277860596704
-0.16908371256963187
-0.16908078067627108
-0.16904474158016441
-0.1690119640320637
-0.16903317301080065
-0.16908783999483593
-0.16908019793583906
-0.16904027123721596
-0.16900666759843555
-0.1690385994000827
-0.1690927820263327
-0.16907709741615112
-0.1690318157474047
-0.16900365578740226
-0.16904872400308224
-0.1690965666661481
-0.16907120739395373
-0.16902491372609826
-0.16900540285625293
-0.169061508441881
-0.169097329523278
-0.16906445026910158
-0.1690197760192123
-0.16900878373305311
-0.16906942996093013
-0.16909369812505687
-0.16905776550851234
-0.16901607122932433
-0.16901202092862033
-0.16907695909886125
-0.16909176966914888
-0.16905402040532883
-0.16901235909924245
-0.16901736033371925
-0.1690847459685637
-0.16909035534674052
-0.16904838452877755
-0.16900663034074542
-0.1690